# Spam Classification

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from pygam import LogisticGAM, s, f
from ast import literal_eval

2023-02-18 23:05:28.570644: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [368]:
import random

In [3]:
#Importing the dataset
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data'
column_names = ["word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d", 
                "word_freq_our", "word_freq_over", "word_freq_remove", "word_freq_internet", 
                "word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will", 
                "word_freq_people", "word_freq_report", "word_freq_addresses", "word_freq_free", 
                "word_freq_business", "word_freq_email", "word_freq_you", "word_freq_credit", 
                "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money", "word_freq_hp", 
                "word_freq_hpl", "word_freq_george", "word_freq_650", "word_freq_lab", "word_freq_labs", 
                "word_freq_telnet", "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85", 
                "word_freq_technology", "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct", 
                "word_freq_cs", "word_freq_meeting", "word_freq_original", "word_freq_project", "word_freq_re", 
                "word_freq_edu", "word_freq_table", "word_freq_conference", "char_freq_;", 
                "char_freq_(", "char_freq_[", "char_freq_!", "char_freq_$", "char_freq_#", 
                "capital_run_length_average", "capital_run_length_longest", "capital_run_length_total", "Spam"]

data = pd.read_csv(url, names=column_names, sep=',',skipinitialspace=True)

In [4]:
#Check for missing values
data.isna().sum()

word_freq_make                0
word_freq_address             0
word_freq_all                 0
word_freq_3d                  0
word_freq_our                 0
word_freq_over                0
word_freq_remove              0
word_freq_internet            0
word_freq_order               0
word_freq_mail                0
word_freq_receive             0
word_freq_will                0
word_freq_people              0
word_freq_report              0
word_freq_addresses           0
word_freq_free                0
word_freq_business            0
word_freq_email               0
word_freq_you                 0
word_freq_credit              0
word_freq_your                0
word_freq_font                0
word_freq_000                 0
word_freq_money               0
word_freq_hp                  0
word_freq_hpl                 0
word_freq_george              0
word_freq_650                 0
word_freq_lab                 0
word_freq_labs                0
word_freq_telnet              0
word_fre

In [5]:
#Split into training and testing data
training_data, testing_data = train_test_split(data, test_size = 0.2, random_state = 5)

In [6]:
training_features = training_data.copy()
training_classes = training_features.pop('Spam')
testing_features = testing_data.copy()
testing_classes = testing_features.pop('Spam')

In [7]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
word_freq_make,4601.0,0.104553,0.305358,0.0,0.000,0.000,0.000,4.540
word_freq_address,4601.0,0.213015,1.290575,0.0,0.000,0.000,0.000,14.280
word_freq_all,4601.0,0.280656,0.504143,0.0,0.000,0.000,0.420,5.100
word_freq_3d,4601.0,0.065425,1.395151,0.0,0.000,0.000,0.000,42.810
word_freq_our,4601.0,0.312223,0.672513,0.0,0.000,0.000,0.380,10.000
word_freq_over,4601.0,0.095901,0.273824,0.0,0.000,0.000,0.000,5.880
word_freq_remove,4601.0,0.114208,0.391441,0.0,0.000,0.000,0.000,7.270
word_freq_internet,4601.0,0.105295,0.401071,0.0,0.000,0.000,0.000,11.110
word_freq_order,4601.0,0.090067,0.278616,0.0,0.000,0.000,0.000,5.260
word_freq_mail,4601.0,0.239413,0.644755,0.0,0.000,0.000,0.160,18.180


# Deep neural network

In [601]:
#58 input variables so 58 input neurons?
#1 output for the set of inputs so 1 output neuron
#hidden layers?

#specify optimiser such as nesterov momentum?

dnn_spam_model = tf.keras.Sequential([
    tf.keras.Input(shape = (57)),
    layers.Dense(36, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation = 'sigmoid')
])
#sigmoid outputs between 0 and 1 which is perfect for probabilities

dnn_spam_model.summary()


Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_123 (Dense)           (None, 36)                2088      
                                                                 
 dense_124 (Dense)           (None, 32)                1184      
                                                                 
 dense_125 (Dense)           (None, 1)                 33        
                                                                 
Total params: 3,305
Trainable params: 3,305
Non-trainable params: 0
_________________________________________________________________


In [602]:
#adam optimiser for advanced gradient descent, loss is binary cross entropy since binary classification
#problem
random.seed(5)
dnn_spam_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                  loss = tf.keras.losses.BinaryCrossentropy(),
                  metrics = [tf.keras.metrics.BinaryAccuracy(name = 'acc')])

In [603]:
#fit model with 100 epochs
dnn_fit = dnn_spam_model.fit(x= training_features, 
               y= training_classes, 
               epochs = 100, 
               validation_split = 0.2,
               class_weight = {0:10, 1:1}
                )
#maybe write code to generate the best number of epochs and learning rate?

Epoch 1/100
92/92 [==============================] - 1s 3ms/step - loss: 8.2329 - acc: 0.6736 - val_loss: 0.9144 - val_acc: 0.7812
Epoch 2/100
92/92 [==============================] - 0s 1ms/step - loss: 5.1279 - acc: 0.7245 - val_loss: 0.4095 - val_acc: 0.8220
Epoch 3/100
92/92 [==============================] - 0s 1ms/step - loss: 2.1238 - acc: 0.7310 - val_loss: 1.1228 - val_acc: 0.7310
Epoch 4/100
92/92 [==============================] - 0s 1ms/step - loss: 1.2124 - acc: 0.7340 - val_loss: 1.1649 - val_acc: 0.6970
Epoch 5/100
92/92 [==============================] - 0s 1ms/step - loss: 0.9790 - acc: 0.7670 - val_loss: 0.3703 - val_acc: 0.8125
Epoch 6/100
92/92 [==============================] - 0s 1ms/step - loss: 1.1198 - acc: 0.7748 - val_loss: 0.2979 - val_acc: 0.8601
Epoch 7/100
92/92 [==============================] - 0s 1ms/step - loss: 0.8724 - acc: 0.7775 - val_loss: 0.3528 - val_acc: 0.8533
Epoch 8/100
92/92 [==============================] - 0s 1ms/step - loss: 1.0669 - a

92/92 [==============================] - 0s 1ms/step - loss: 0.5139 - acc: 0.8838 - val_loss: 0.3578 - val_acc: 0.8696
Epoch 64/100
92/92 [==============================] - 0s 1ms/step - loss: 0.3955 - acc: 0.8961 - val_loss: 0.2900 - val_acc: 0.9049
Epoch 65/100
92/92 [==============================] - 0s 1ms/step - loss: 0.3660 - acc: 0.9039 - val_loss: 0.2785 - val_acc: 0.9117
Epoch 66/100
92/92 [==============================] - 0s 1ms/step - loss: 0.3972 - acc: 0.9086 - val_loss: 0.2686 - val_acc: 0.9090
Epoch 67/100
92/92 [==============================] - 0s 1ms/step - loss: 0.3647 - acc: 0.9090 - val_loss: 0.2859 - val_acc: 0.9158
Epoch 68/100
92/92 [==============================] - 0s 2ms/step - loss: 0.3446 - acc: 0.9151 - val_loss: 0.2702 - val_acc: 0.9253
Epoch 69/100
92/92 [==============================] - 0s 1ms/step - loss: 0.4130 - acc: 0.8859 - val_loss: 0.3262 - val_acc: 0.8832
Epoch 70/100
92/92 [==============================] - 0s 1ms/step - loss: 0.7003 - acc: 0

In [ ]:
#Check the training progress/error by epoch, store in dataframe
history = pd.DataFrame(dnn_fit.history)
history['epoch'] = dnn_fit.epoch
history

In [ ]:
plt.plot(
    np.arange(1, 91), 
    dnn_fit.history['acc'], 
    label='Accuracy'
)

plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

In [609]:
#Moving onto predicting
dnn_test_pred = dnn_spam_model.predict(testing_features)
dnn_pred_classes = [
    1 if prob > 0.9993 else 0 for prob in np.ravel(dnn_test_pred)
]
print(confusion_matrix(testing_classes, dnn_pred_classes))

print(f'Accuracy: {accuracy_score(testing_classes, dnn_pred_classes)}')


29/29 [==============================] - 0s 852us/step
[[575   0]
 [234 112]]
Accuracy: 0.745928338762215


# Classification GAM

In [17]:
classifier = LogisticGAM()
classifier.fit(training_features, training_classes)

/Users/Cheks/opt/anaconda3/lib/python3.9/site-packages/pygam/pygam.py:752: RuntimeWarning: invalid value encountered in reciprocal
  np.fill_diagonal(Dinv, d**-1) # invert the singular values
/Users/Cheks/opt/anaconda3/lib/python3.9/site-packages/pygam/links.py:149: RuntimeWarning: divide by zero encountered in true_divide
  return dist.levels/(mu*(dist.levels - mu))
/Users/Cheks/opt/anaconda3/lib/python3.9/site-packages/pygam/pygam.py:591: RuntimeWarning: invalid value encountered in multiply
  return sp.sparse.diags((self.link.gradient(mu, self.distribution)**2 *


LogisticGAM(callbacks=[Deviance(), Diffs(), Accuracy()], 
   fit_intercept=True, max_iter=100, 
   terms=s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10) + s(11) + s(12) + s(13) + s(14) + s(15) + s(16) + s(17) + s(18) + s(19) + s(20) + s(21) + s(22) + s(23) + s(24) + s(25) + s(26) + s(27) + s(28) + s(29) + s(30) + s(31) + s(32) + s(33) + s(34) + s(35) + s(36) + s(37) + s(38) + s(39) + s(40) + s(41) + s(42) + s(43) + s(44) + s(45) + s(46) + s(47) + s(48) + s(49) + s(50) + s(51) + s(52) + s(53) + s(54) + s(55) + s(56) + intercept,
   tol=0.0001, verbose=False)

In [18]:
gam_pred_classes = classifier.predict(testing_features)
gam_pred_classes = pred_classes * 1

NameError: name 'pred_classes' is not defined

In [ ]:
classifier.summary()

In [ ]:
print(confusion_matrix(testing_classes, gam_pred_classes))
